In [33]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [34]:
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
from my_code import pytorch_model as m
from my_code import quantum_nodes as q

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# name of the notebook
name_notebook = "sweep_6h_er_1qm_oi"

In [36]:
# sweep = c.Sweep(
#     name_notebook, 
#     initial_path=initial_path,
#     description="(6aa, 6q) (generate inputs) Try to find best number of q. layers, embedding dim.",
#     repetitions=list(range(1, 4)),
#     embedding_dim=[2,3,4,5],
#     block_n_layers=[3,5,7,9,11,13],
# )

In [37]:
# sweep.save()

In [38]:
sweep = c.Sweep.load(file_name=name_notebook+"-1", day='1002', initial_path=initial_path)

In [39]:
for sweep_point in sweep.points_left:

    # print the point of the sweep
    sweep.print_sweep_status(sweep_point['idx'])

    # -- QUANTUM BLOCK -- #
    n_qubits = 6
    quantum_layer = q.circuit(
        n_qubits = n_qubits, 
        device = "default.qubit.torch",
        device_options = {'shots': None},
        embedding = q.parts.AngleEmbedding,
        embedding_ansatz = q.parts.Ansatz_11,
        block_ansatz = q.parts.Ansatz_11,
        final_ansatz = q.parts.Ansatz_11, 
        measurement = q.parts.Measurement('Z', 1),
        embedding_n_layers = sweep_point['embedding_dim']-1,
        different_inputs_per_layer = True,
        block_n_layers = sweep_point['block_n_layers'],
        # wrapper_qlayer = pw.QLayerEmpty,
    )

    # -- MODEL -- #                    
    class Model(m.Model):
        def __init__(self):
            super(Model, self).__init__()
            self.fc1 = m.nn.Embedding(num_embeddings=19, embedding_dim=sweep_point['embedding_dim'])
            self.fc2 = m.Reshape()
            self.fc3 = quantum_layer()            
            self.fc4 = m.nn.Linear(1, 1)

        def forward(self, x):
            for i in [1,2,3,4]:
                x = getattr(self, 'fc{}'.format(i))(x)
            return x

    model = Model()

    # set quantum layer
    model.set_quantum_layer(quantum_layer)

    # set the name and path of the model
    model.set_name_and_path(name_notebook, initial_path)

    #set sweep point
    model.set_sweep_point(
        sweep_uuid = sweep.uuid,
        sweep_point = sweep_point,
        day=sweep.day
    )

    # -- TRAIN -- #
    model.optimize_params(
        #save info
        save_model_info = True,
        description = 'sweep with 6 aa and 6 q -> Try to find best number of q. layers, measurement, and type of Anzatz.',    
        #data
        data=c.Data.load(initial_path=initial_path, file_name='PE_SCORES_6_Numbers'),
        # train options
        optimizer = m.optim.Adam,
        optimizer_options = {'lr': 0.01},
        num_epochs = 20,                
        batch_size = 32,
        # early stopping options
        stop_training_options = {
            'lookback_epochs': 5, 
            'threshold_slope': 0.001,
            'threshold_std_dev': 0.2
        },
        # metadata
        metadata = {}
    )

    # load best model
    model.load_state_dict_from_results(state_dict_name="parameters_epoch", rule="best")

    # -- VALIDATION and PLOTS -- #
    model.plot_losses(fig_size=(6,6))
    model.validate(
        pct=1,
        add_to_results=True,
        plot=True,
        print_items=False
    )

    # -- SAVE RESULTS -- #
    model.save_results()

    # -- GENERATE PEPTIDES and SAVE THEM -- #    
    oi_time = m.t.time()
    n_peptides = 20
    for i in range(n_peptides):
        model.get_input_with_low_score_2(
            layers_to_use = ['fc3', 'fc4'],
            n_iter = 1000,
            optimizer = m.torch.optim.Adam,
            optimizer_options = {'lr': 0.1},
            stop_criterion = 1e-3,
            initialization_seed = i,
        )
        print("{}/{} peptides generated. Score {}. \t Time remaining: {:.2f} min \t\t\t ".format(
            i+1, n_peptides, model.optimized_inputs[-1][1], (n_peptides-i)*(m.t.time() - oi_time)/(i+1)/60
        ), end='\r')
    oi_time = m.t.time() - oi_time
    model.save_input_optimization_results()
    best_oi, best_oi_score = model.best_optimized_input
    print("Best score: {} \t\t\t\t\t\t  ".format(best_oi_score))

    # -- ADD DATA GENERATED TO SWEEP -- #
    sweep.add_data(
        idx=sweep_point['idx'],
        model_uuid=model.uuid,
        best_loss_train=model.results.loss_epoch.best,
        best_loss_test=model.results.loss_validation_epoch.best,
        loss_validation=model.mean_loss_validation,
        loss_validation_r_squared=model.results.validation['r_squared'],
        n_parameters=model.n_parameters,
        time_train=model.results.time_epoch.last,
        best_oi=best_oi,
        best_oi_score=best_oi_score,
        oi_time=oi_time,
    )
    sweep.save(csv=False) #TODO: save csv=False automatically when sweep is already saved ion csv

    



 --- SWEEP POINT 1/72: (1, 2, 3) ---
 --- parameters sweeping: ['repetitions', 'embedding_dim', 'block_n_layers'] 

Epoch [0/20], Loss epoch: 154.8372, Loss validation: 155.7426
	 Validation string, 	 i: 51; 	 prediction: -6.1748, 	 target: -17.7400, 	 loss: 133.7548
	 Validation string, 	 i: 107; 	 prediction: -6.1747, 	 target: -8.3800, 	 loss: 4.8632
	 Validation string, 	 i: 37; 	 prediction: -6.1747, 	 target: -6.6100, 	 loss: 0.1895
Epoch [1/20], Loss epoch: 86.5787, Loss validation: 46.5750, Time remaining: ~0.0h 9.0m 21s
	 Validation string, 	 i: 43; 	 prediction: -8.7910, 	 target: -5.4900, 	 loss: 10.8967
	 Validation string, 	 i: 70; 	 prediction: -8.3245, 	 target: -15.6800, 	 loss: 54.1033
	 Validation string, 	 i: 74; 	 prediction: -8.2802, 	 target: -8.8000, 	 loss: 0.2702
Epoch [2/20], Loss epoch: 44.6719, Loss validation: 33.2474, Time remaining: ~0.0h 8.0m 30s
	 Validation string, 	 i: 16; 	 prediction: -9.8345, 	 target: -16.3100, 	 loss: 41.9316
	 Validation strin